In [63]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv


df = pd.read_csv('../Datasets/processed_data.csv')
df_advertiser = pd.read_csv('../Datasets/advertiser_embeddings.csv')
df_keywords = pd.read_csv('../Datasets/keyword_embeddings.csv')
df_search_tags = pd.read_csv('../Datasets/search_tag_embeddings.csv')



In [64]:
df = pd.concat([df, df_advertiser, df_keywords, df_search_tags], axis=1)


In [65]:
import torch
from torch_geometric.data import HeteroData

# Initialize graph
data = HeteroData()

# Node ID mappings
campaign_mapping = {}  # {campaign_id: index}
platform_mapping = {}  # {platform_id: index}
channel_mapping = {}  # {channel_id: index}
advertiser_mapping = {}  # {advertiser_word: index}
keywords_mapping = {}  # {keyword: index}
search_tags_mapping = {}  # {search_tag: index}



# Unique ID counters
campaign_counter = 0
platform_counter = 0
channel_counter = 0
advertiser_counter = 0
keywords_counter = 0
search_tags_counter = 0


In [66]:
def update_mapping(mapping, key, counter):
    """Ensure new inputs are added to the mapping with unique indices."""
    if key not in mapping:
        mapping[key] = counter
        counter += 1
    return mapping[key], counter

In [67]:
# Remove rows with any NaN values in any column
df = df.dropna()

## Campaign Nodes
### Each campaign treated as single entity across different rows

In [68]:
campaign_features = []  
campaign_counter = len(campaign_mapping)  # Start from existing mappings

for _, row in df[['campaign_index', 'no_of_days']].iterrows():
    campaign_id = row['campaign_index']
    features = row['no_of_days']

    campaign_id, campaign_counter = update_mapping(campaign_mapping, campaign_id, campaign_counter)
    campaign_features.append([features])  

data['campaign'].x = torch.tensor(campaign_features, dtype=torch.float)


## Platform Node

In [69]:
platform_data = df[['DV360', 'Facebook Ads', 'Google Ads']]
# Convert to tensor format
platform_features = torch.tensor(platform_data.values, dtype=torch.float)
# data['platform'] is the platform node feature storage
data['platform'].x = platform_features

# Assign unique indices
platform_mapping = {'DV360': 0, 'Google Ads': 1, 'Facebook Ads': 2}


## Channel  Node

In [70]:
channel_data = df[['Display','Mobile','Search','Social','Video']].values
# Convert to tensor format
channel_features = torch.tensor(channel_data, dtype=torch.float)
data['channel'].x = channel_features

# Assign unique indices
channel_mapping = {'Display': 4,'Mobile': 0, 'Search': 1, 'Social': 2, 'Video': 3}


## Advertiser Node

In [71]:
advertiser_mapping = {}  # Initialize the mapping
advertiser_counter = 0  # Initialize the counter for unique advertiser indices

advertiser_features = []  # Store advertiser embeddings

for _, row in df[['advertiser_name'] + [f'advertiser_emb_{i}' for i in range(50)]].iterrows():
    advertiser_name = str(row['advertiser_name'])  # Convert to string to avoid "unhashable type: Series"
    embedding = row[[f'advertiser_emb_{i}' for i in range(50)]].values  

    # Ensure the advertiser is mapped and assign a unique index
    if advertiser_name not in advertiser_mapping:
        advertiser_mapping[advertiser_name] = advertiser_counter
        advertiser_counter += 1

    advertiser_features.append(embedding)

# Store the embeddings in the data structure
data['advertiser'].x = torch.tensor(advertiser_features, dtype=torch.float)


In [72]:
print(data['advertiser'].x.shape)  # Expected: (58930, 50)


torch.Size([58930, 50])


## Keywords Node

In [73]:
keyword_features = []  
keywords_counter = len(keywords_mapping)  

for _, row in df[['keyword'] + [f'keyword_emb_{i}' for i in range(50)]].iterrows():
    keyword = row['keyword']
    embedding = row[[f'keyword_emb_{i}' for i in range(50)]].values  

    keyword, keywords_counter = update_mapping(keywords_mapping, keyword, keywords_counter)
    keyword_features.append(embedding)

data['keyword'].x = torch.tensor(keyword_features, dtype=torch.float)


## Creative Node

In [74]:
creative_data = df[['has_image']]
# Convert to tensor format
creative_features = torch.tensor(creative_data.values, dtype=torch.float)

# Assign to creative nodes
data['creative'].x = creative_features  # Assign binary feature


In [75]:
print(data['creative'].x.shape)  # Expected: (58930, 1)

torch.Size([58930, 1])


## Search tag Node

In [76]:
search_tag_features = []  
search_tags_counter = len(search_tags_mapping)  

for _, row in df[['search_tag'] + [f'search_tag_emb_{i}' for i in range(50)]].iterrows():
    search_tag = row['search_tag']
    embedding = row[[f'search_tag_emb_{i}' for i in range(50)]].values  

    search_tag, search_tags_counter = update_mapping(search_tags_mapping, search_tag, search_tags_counter)
    search_tag_features.append(embedding)

data['search_tag'].x = torch.tensor(search_tag_features, dtype=torch.float)


### ➤ Campaign → Platform Edges

In [77]:
campaign_platform_edges = []
campaign_platform_data = []


# Iterate over your dataset (df_filtered) and get campaign_id and platform
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    platform_name = row['platform']   # Get platform name
    campaign_platform_data.append([campaign_id, platform_name])  # Add to list

for campaign_id, platform_name in campaign_platform_data:  # (campaign_id, platform_name)
    campaign_idx = campaign_mapping[campaign_id]
    platform_idx = platform_mapping[platform_name]

    campaign_platform_edges.append([campaign_idx, platform_idx])

platform_campaign_edges = [[platform_idx, campaign_idx] for campaign_idx, platform_idx in campaign_platform_edges]
data['platform', 'rev_hosted_on', 'campaign'].edge_index = torch.tensor(platform_campaign_edges, dtype=torch.long).T
data['campaign', 'hosted_on', 'platform'].edge_index = torch.tensor(campaign_platform_edges, dtype=torch.long).T


### ➤ Campaign → Channel Edges

In [78]:
campaign_channel_edges = []
campaign_channel_data = []

# Iterate over your dataset (df_filtered) and get campaign_id and channel
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    channel_name = row['channel_name']   # Get channel name
    campaign_channel_data.append([campaign_id, channel_name])  # Add to list

for campaign_id, channel_name in campaign_channel_data:  # (campaign_id, channel_name)
    campaign_idx = campaign_mapping[campaign_id]
    channel_idx = channel_mapping[channel_name]

    campaign_channel_edges.append([campaign_idx, channel_idx])

channel_campaign_edges = [[channel_idx, campaign_idx] for campaign_idx, channel_idx in campaign_channel_edges]
data['channel', 'rev_uses', 'campaign'].edge_index = torch.tensor(channel_campaign_edges, dtype=torch.long).T
data['campaign', 'uses', 'channel'].edge_index = torch.tensor(campaign_channel_edges, dtype=torch.long).T


### ➤ Campaign → Advertiser Edges

In [79]:
campaign_advertiser_edges = []
campaign_advertiser_data = []

# Iterate over your dataset (df_filtered) and get campaign_id and advertiser_name
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    advertiser_name = str(row['advertiser_name'])
    campaign_advertiser_data.append([campaign_id, advertiser_name])  # Add to list

for campaign_id, advertiser_name in campaign_advertiser_data:  # (campaign_id, advertiser_name)
    campaign_idx = campaign_mapping[campaign_id]  # Ensure campaign mapping is valid
    if advertiser_name not in advertiser_mapping:
        raise KeyError(f"Advertiser '{advertiser_name}' not found in advertiser mapping.")
    advertiser_idx = advertiser_mapping[advertiser_name]  # Map advertiser name to index
    campaign_advertiser_edges.append([campaign_idx, advertiser_idx])

advertiser_campaign_edges = [[advertiser_idx, campaign_idx] for campaign_idx, advertiser_idx in campaign_advertiser_edges]
data['advertiser', 'rev_belongs_to', 'campaign'].edge_index = torch.tensor(advertiser_campaign_edges, dtype=torch.long).T
data['campaign', 'belongs_to', 'advertiser'].edge_index = torch.tensor(campaign_advertiser_edges, dtype=torch.long).T


### ➤ Campaign → keywords Edges

In [80]:
campaign_keywords_edges = []
campaign_keywords_data = []

# Iterate over your dataset (df_filtered) and get campaign_id and keywords
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    keywords = row['keywords']   # Get keywords name
    campaign_keywords_data.append([campaign_id, keywords])  # Add to list

for campaign_id, keywords in campaign_keywords_data:  # (campaign_id, keywords)
    campaign_idx = campaign_mapping[campaign_id]
    keywords_idx = keywords_mapping[keywords]
    campaign_keywords_edges.append([campaign_idx, keywords_idx])

keywords_campaign_edges = [[keywords_idx, campaign_idx] for campaign_idx, keywords_idx in campaign_keywords_edges]
data['keyword', 'rev_associated_with', 'campaign'].edge_index = torch.tensor(keywords_campaign_edges, dtype=torch.long).T
data['campaign', 'associated_with', 'keyword'].edge_index = torch.tensor(campaign_keywords_edges, dtype=torch.long).T


### ➤ Campaign → Creative Edges

In [81]:
campaign_creates_edges = []
campaign_creates_data = []

# Iterate over your dataset (df_filtered) and get campaign_id and creative
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    creative = row['has_image']   # Get creative
    campaign_creates_data.append([campaign_id, creative])  # Add to list

for campaign_id, creative in campaign_creates_data:  # (campaign_id, creative)
    campaign_idx = campaign_mapping[campaign_id]

    campaign_creates_edges.append([campaign_idx, creative])

creative_campaign_edges = [[creative, campaign_idx] for campaign_idx, creative in campaign_creates_edges]
data['creative', 'rev_uses', 'campaign'].edge_index = torch.tensor(creative_campaign_edges, dtype=torch.long).T
data['campaign', 'uses', 'creative'].edge_index = torch.tensor(campaign_creates_edges, dtype=torch.long).T

### ➤ Platform → Channel Edges

In [82]:
platform_channel_data = {
    'DV360': ['Display', 'Mobile', 'Search', 'Social', 'Video'],
    'Google Ads': ['Display', 'Mobile', 'Search', 'Social', 'Video'],
    'Facebook Ads': ['Display', 'Mobile', 'Search', 'Social', 'Video']
}

# List to store platform-channel edges
platform_channel_edges = []

# Iterate over the platform-channel relationships
for platform, channels in platform_channel_data.items():
    # Get the platform index
    platform_idx = platform_mapping[platform]
    
    # Iterate over the channels associated with this platform
    for channel in channels:
        # Get the channel index
        channel_idx = channel_mapping[channel]
        
        # Add the edge (platform -> channel)
        platform_channel_edges.append([platform_idx, channel_idx])

# Convert to tensor and assign to the graph
data['platform', 'supports', 'channel'].edge_index = torch.tensor(platform_channel_edges, dtype=torch.long).T


### ➤ Campaign → Search tag Edges

In [83]:
campaign_searchtag_edges = []
campaign_searchtag_data = []

# Iterate over your dataset (df_filtered) and get campaign_id and keywords
for index, row in df.iterrows():
    campaign_id = row['campaign_index']  # Get campaign_id
    search_tag = row['search_tags']   # Get keywords name
    campaign_searchtag_data.append([campaign_id, search_tag])  # Add to list

for campaign_id, keywords in campaign_searchtag_data:  # (campaign_id, keywords)
    campaign_idx = campaign_mapping[campaign_id]
    search_tag_idx = search_tags_mapping[search_tag]
    campaign_searchtag_edges.append([campaign_idx, search_tag_idx])

searchtag_campaign_edges = [[search_tag_idx, campaign_idx] for campaign_idx, search_tag_idx in campaign_searchtag_edges]
data['search_tag', 'rev_targeted_by', 'campaign'].edge_index = torch.tensor(searchtag_campaign_edges, dtype=torch.long).T
data['campaign', 'targeted_by', 'search_tag'].edge_index = torch.tensor(campaign_searchtag_edges, dtype=torch.long).T


### ➤ Platform → Keywords Edges

In [84]:
import json

# Load the data from the JSON file
with open('../Datasets/platform_keyword_data.json', 'r') as f:
    platform_keyword_data = json.load(f)

print(platform_keyword_data)  # You can now use this data in your program


{'Facebook Ads': ['affordable jewelry', 'elegant jewelry', 'seashell jewelry', 'cuff bracelets', 'holiday jewelry', 'unique jewelry', 'dazzling jewelry', 'jewelry sets', 'boho jewelry', 'statement jewelry', 'cocktail rings', 'beaded jewelry', 'bold jewelry', 'toe rings', 'vintage-inspired jewelry', 'handcrafted jewelry', 'ear cuffs', 'zodiac jewelry', 'formal jewelry', 'mixed metal jewelry', 'choker necklaces', 'exquisite jewelry', 'pearl jewelry', 'wedding jewelry', 'dainty jewelry', 'whimsical jewelry', 'bangles', 'affordable statement jewelry', 'stackable bracelets', 'artisan jewelry', 'fall jewelry', 'fashion brooches', 'minimalistic jewelry', 'pendant necklaces', 'layered bracelets', 'hoop earrings', 'anklets', 'charm bracelets', 'handmade jewelry', 'retro jewelry', 'initial jewelry', 'casual chic jewelry', 'threader earrings', 'nature-inspired jewelry', 'artisanal jewelry', 'fashion jewelry for women', 'body chains', 'animal jewelry', 'beaded bracelets', 'festival jewelry', 'pers

In [85]:
platform_keyword_edges = []

# Iterate over the platform-to-keyword relationships
for platform, keywords in platform_keyword_data.items():
    # Get the platform index
    platform_idx = platform_mapping[platform]

    # Iterate over the associated keywords
    for keyword in keywords:
        # Get the keyword index
        keyword_idx = keywords_mapping[keyword]

        # Add the edge (platform -> keyword)
        platform_keyword_edges.append([platform_idx, keyword_idx])

# Convert to tensor and assign to the graph
data['platform', 'optimized_for', 'keyword'].edge_index = torch.tensor(platform_keyword_edges, dtype=torch.long).T


In [86]:
# Remove rows where engagement_level is NaN
df = df.dropna(subset=['engagement_level'])

# Extract engagement levels for each unique campaign-feature instance (row-wise)
engagement_labels = torch.tensor(df['engagement_level'].values, dtype=torch.long)

# Assign each instance's engagement level as its target
data['campaign'].y = engagement_labels


In [87]:
# print(f"Campaign features shape: {campaign_features.shape}")  # Should be (58930, feature_dim)
print(f"Campaign labels shape: {data['campaign'].y.shape}")  # Should match the number of campaign features

Campaign labels shape: torch.Size([58930])


In [88]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, HeteroConv

class EngagementGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels):
        super(EngagementGNN, self).__init__()

        # Convert all node features to the same size
        self.feature_transforms = torch.nn.ModuleDict({
            'campaign': torch.nn.Linear(1, hidden_channels),  # Project scalar feature
            'platform': torch.nn.Linear(3, hidden_channels),  # Project one-hot to hidden size
            'channel': torch.nn.Linear(5, hidden_channels),
            'advertiser': torch.nn.Linear(50, hidden_channels),
            'keyword': torch.nn.Linear(50, hidden_channels),
            'creative': torch.nn.Linear(1, hidden_channels),  # Project scalar feature
            'search_tag': torch.nn.Linear(50, hidden_channels),
        })
        
        self.hidden_channels = hidden_channels
        self.metadata = metadata  # Store metadata for use in forward()
        
        # Create GAT layers for each edge type
        self.convs = HeteroConv({
            edge_type: GATConv(in_channels, hidden_channels, heads=4, concat=True, add_self_loops=False)
            for edge_type, (in_channels, _) in metadata['edge_dim'].items()
        })
        
        # Final linear layer for classification
        self.out = torch.nn.ModuleDict({
            node_type: torch.nn.Linear(hidden_channels * 4, out_channels)
            for node_type in metadata['node_types']
        })


    def forward(self, x_dict, edge_index_dict):
        
        # Apply feature transformation to each node type
        x_dict = {node: self.feature_transforms[node](feat) for node, feat in x_dict.items()}
        # Apply GAT layers on heterogeneous graph
        x_dict = self.convs(x_dict, edge_index_dict)
        out_dict = {node: self.out[node](x_dict[node]) for node in x_dict}

        return out_dict


In [89]:
def compute_accuracy(predictions, labels):
    _, predicted = torch.max(predictions, dim=1)  # Get the predicted class
    correct = (predicted == labels).sum()  # Count the number of correct predictions
    accuracy = correct / len(labels)  # Accuracy as a fraction
    return accuracy.item()  # Return as a scalar value

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import torch

# Define training function
def train(model, data, optimizer, criterion, epochs=100):
    all_predictions = []
    all_labels = []
    loss_values = []
    accuracy_values = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Collect node features dynamically
        node_features = {node_type: data[node_type].x for node_type in data.node_types}

        # Collect edge indices dynamically
        edge_index_dict = {edge_type: data[edge_type].edge_index for edge_type in data.edge_types}

        # Forward pass with node features & edge index
        out_dict = model(node_features, edge_index_dict)

        total_loss = 0
        total_accuracy = 0
        count = 0

        for node_type in data.node_types:
            if hasattr(data[node_type], 'y'):  # Ensure the node type has labels
                node_output = out_dict[node_type]  # Get predictions for the node type
                node_labels = data[node_type].y  # True labels

                loss = criterion(node_output, node_labels)
                total_loss += loss

                # Compute accuracy for this node type
                accuracy = compute_accuracy(node_output, node_labels)
                total_accuracy += accuracy
                count += 1

                # Get predictions and labels for classification report
                predictions = torch.argmax(node_output, dim=1).cpu().numpy()
                labels = node_labels.cpu().numpy()

                all_predictions.extend(predictions)
                all_labels.extend(labels)

        # Backpropagation
        total_loss.backward()
        optimizer.step()

        # Average loss and accuracy
        avg_loss = total_loss.item() / count
        avg_accuracy = total_accuracy / count

        loss_values.append(avg_loss)
        accuracy_values.append(avg_accuracy)

        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.4f}")

    # Final classification report
    print("\nFinal Classification Report:")
    print(classification_report(all_labels, all_predictions))

    # Plot loss and accuracy over epochs
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(epochs), loss_values, label='Loss', color='blue')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(range(epochs), accuracy_values, label='Accuracy', color='green')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()


: 

In [ ]:
node_types, edge_types = data.metadata()  # Unpack the tuple

# Check output to confirm
print("Node Types:", node_types)  # List of nodes
print("Edge Types:", edge_types)  # List of edges

# Now manually construct metadata the way the model expects
metadata = {
    'node_types': node_types,
    'edge_types': edge_types,
    'edge_dim': {edge: (64, 8) for edge in edge_types}  # Example, adjust dimensions as needed
}

# Now it should work
model = EngagementGNN(metadata=metadata, hidden_channels=64, out_channels=2)

# Optimizer & Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

# Train the model
train(model, data, optimizer, criterion, epochs=100)


Node Types: ['campaign', 'platform', 'channel', 'advertiser', 'keyword', 'creative', 'search_tag']
Edge Types: [('platform', 'rev_hosted_on', 'campaign'), ('campaign', 'hosted_on', 'platform'), ('channel', 'rev_uses', 'campaign'), ('campaign', 'uses', 'channel'), ('advertiser', 'rev_belongs_to', 'campaign'), ('campaign', 'belongs_to', 'advertiser'), ('keyword', 'rev_associated_with', 'campaign'), ('campaign', 'associated_with', 'keyword'), ('creative', 'rev_uses', 'campaign'), ('campaign', 'uses', 'creative'), ('platform', 'supports', 'channel'), ('search_tag', 'rev_targeted_by', 'campaign'), ('campaign', 'targeted_by', 'search_tag'), ('platform', 'optimized_for', 'keyword')]
Epoch 1/100, Loss: 0.6931, Accuracy: 0.5131
Epoch 2/100, Loss: 0.6930, Accuracy: 0.5132
Epoch 3/100, Loss: 0.6929, Accuracy: 0.5131
Epoch 4/100, Loss: 0.6929, Accuracy: 0.5131
Epoch 5/100, Loss: 0.6928, Accuracy: 0.5132
Epoch 6/100, Loss: 0.6928, Accuracy: 0.5132
Epoch 7/100, Loss: 0.6928, Accuracy: 0.5132
Epoch 8

In [ ]:
import torch
import numpy as np

def process_user_input(user_input):
    """
    Convert user inputs into numerical tensors for prediction.
    
    Args:
    user_input (dict): Dictionary containing user-provided values.

    Returns:
    dict: A dictionary with tensors formatted for model input.
    """

    # 🔹 Retrieve Campaign Index (Ensure it's valid)
    campaign_idx = campaign_mapping.get(user_input['campaign'], None)
    if campaign_idx is None:
        raise KeyError(f"Campaign ID {user_input['campaign']} not found in mapping!")

    # 🔹 Campaign Features (MUST include campaign index + no_of_days)
    campaign_features = [campaign_idx, user_input['no_of_days']]  # FIXED

    # 🔹 One-hot encode platform
    platform_vector = [0, 0, 0]
    if user_input['platform'] in platform_mapping:
        platform_vector[platform_mapping[user_input['platform']]] = 1
    else:
        print(f"Warning: Platform '{user_input['platform']}' not found in mapping.")

    # 🔹 One-hot encode channel
    channel_vector = [0, 0, 0, 0, 0]
    if user_input['channel'] in channel_mapping:
        channel_vector[channel_mapping[user_input['channel']]] = 1
    else:
        print(f"Warning: Channel '{user_input['channel']}' not found in mapping.")

    # 🔹 Advertiser embedding (use existing if available, otherwise random)
    advertiser_vector = advertiser_mapping.get(
        user_input['advertiser_name'], np.random.rand(50)
    )

    # 🔹 Keyword embedding (use existing if available, otherwise random)
    keyword_vector = keywords_mapping.get(
        user_input['keyword'], np.random.rand(50)
    )

    # 🔹 Creative feature (binary)
    creative_vector = [user_input['has_image']]

    # 🔹 Search tag embedding (use existing if available, otherwise random)
    search_tag_vector = search_tags_mapping.get(
        user_input['search_tag'], np.random.rand(50)
    )

    # 🔹 Retrieve Node IDs for Edge Index
    platform_idx = platform_mapping.get(user_input['platform'], None)
    channel_idx = channel_mapping.get(user_input['channel'], None)
    advertiser_idx = advertiser_mapping.get(user_input['advertiser_name'], None)
    keyword_idx = keywords_mapping.get(user_input['keyword'], None)

    # Validate all indices before creating edge index
    missing_mappings = []
    if platform_idx is None: missing_mappings.append('platform')
    if channel_idx is None: missing_mappings.append('channel')
    if advertiser_idx is None: missing_mappings.append('advertiser')
    if keyword_idx is None: missing_mappings.append('keyword')

    if missing_mappings:
        raise KeyError(f"One or more input mappings were not found: {', '.join(missing_mappings)}")

    # 🔹 Construct Edge Index (Source -> Target)
    edge_index = torch.tensor([ 
        [campaign_idx, platform_idx],   # Campaign -> Platform
        [campaign_idx, channel_idx],    # Campaign -> Channel
        [campaign_idx, advertiser_idx], # Campaign -> Advertiser
        [campaign_idx, keyword_idx]     # Campaign -> Keyword
    ], dtype=torch.long).T  # Transpose for PyG format

    # Convert all inputs to tensors
    input_tensors = {
        'campaign': torch.tensor([campaign_features], dtype=torch.float),  # FIXED
        'platform': torch.tensor([platform_vector], dtype=torch.float),
        'channel': torch.tensor([channel_vector], dtype=torch.float),
        'advertiser': torch.tensor([advertiser_vector], dtype=torch.float),
        'keyword': torch.tensor([keyword_vector], dtype=torch.float),
        'has_image': torch.tensor([creative_vector], dtype=torch.float),
        'search_tag': torch.tensor([search_tag_vector], dtype=torch.float),
        'edge_index': edge_index
    }

    return input_tensors


In [ ]:
# Example user input (from an API or user form)
user_input = {
    "campaign": 13,  # Categorical input
    "no_of_days": 10,  # Numeric input
    "platform": "Google Ads",  # One-hot encoded
    "channel": "Social",  # One-hot encoded
    "advertiser_name": "Web",  # Mapped to embedding
    "keyword": "bohemian jewelry",  # Mapped to embedding
    "has_image": 1,  # Binary feature
    "search_tag": "Athletic"  # Mapped to embedding
}

# Convert input to numeric tensors
formatted_inputs = process_user_input(user_input)

# Display formatted tensors
for key, value in formatted_inputs.items():
    print(f"{key}: {value.shape}")


KeyError: 'One or more input mappings were not found: advertiser'

In [ ]:
print("Campaign Mapping:", campaign_mapping)
print("Campaign ID from user:", user_input['campaign'])
print("Mapped Campaign Index:", campaign_idx)


Campaign Mapping: {0.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 5.0: 5, 6.0: 6, 9.0: 7, 10.0: 8, 11.0: 9, 13.0: 10, 12.0: 11, 14.0: 12, 15.0: 13, 16.0: 14, 17.0: 15, 18.0: 16, 19.0: 17, 20.0: 18, 21.0: 19, 22.0: 20, 23.0: 21, 24.0: 22, 26.0: 23, 28.0: 24, 30.0: 25, 31.0: 26, 32.0: 27, 33.0: 28, 34.0: 29, 35.0: 30, 36.0: 31, 38.0: 32, 39.0: 33, 40.0: 34, 41.0: 35, 42.0: 36, 43.0: 37, 45.0: 38, 46.0: 39, 7.0: 40, 48.0: 41, 49.0: 42, 51.0: 43, 54.0: 44, 55.0: 45, 56.0: 46, 57.0: 47, 58.0: 48, 59.0: 49, 61.0: 50, 62.0: 51, 60.0: 52, 8.0: 53, 63.0: 54, 64.0: 55, 65.0: 56, 66.0: 57, 67.0: 58, 68.0: 59, 69.0: 60, 70.0: 61, 71.0: 62, 72.0: 63, 73.0: 64, 74.0: 65, 75.0: 66, 76.0: 67, 77.0: 68, 78.0: 69, 79.0: 70, 80.0: 71, 81.0: 72, 82.0: 73, 83.0: 74, 84.0: 75, 85.0: 76, 86.0: 77, 87.0: 78, 88.0: 79, 89.0: 80, 90.0: 81, 91.0: 82, 92.0: 83, 93.0: 84, 94.0: 85, 95.0: 86, 96.0: 87, 97.0: 88, 98.0: 89, 99.0: 90, 100.0: 91, 101.0: 92, 102.0: 93, 103.0: 94, 104.0: 95, 105.0: 96, 106.0: 97, 107.0: 98}
Cam

In [ ]:
print("Platform Mapping:", platform_mapping)
print("User Platform:", user_input['platform'])
print("Mapped Platform Index:", platform_idx)

print("Channel Mapping:", channel_mapping)
print("User Channel:", user_input['channel'])
print("Mapped Channel Index:", channel_idx)


Platform Mapping: {'DV360': 0, 'Google Ads': 1, 'Facebook Ads': 2}
User Platform: Google Ads
Mapped Platform Index: 1
Channel Mapping: {'Display': 4, 'Mobile': 0, 'Search': 1, 'Social': 2, 'Video': 3}
User Channel: Social
Mapped Channel Index: 3


In [ ]:
import matplotlib
print(matplotlib.__version__)


3.9.1


In [ ]:
def predict_engagement(model, input_data):
    """
    Predict engagement level using the trained model.
    
    Args:
    model (torch.nn.Module): Trained GNN model.
    input_data (dict): Dictionary containing tensors of user input features.

    Returns:
    torch.Tensor: Predicted engagement level.
    """
    model.eval()  # Set model to evaluation mode

    with torch.no_grad():  # No need to track gradients
        prediction = model(input_data['campaign'], input_data['edge_index'])  # Pass edge_index

    return prediction

In [ ]:
# Assuming 'gnn_model' is your trained GNN model
prediction = predict_engagement(model, formatted_inputs)

print("Predicted Engagement Level:", prediction.item())


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x2 and 1x128)